In [355]:
import pandas as pd
import numpy as np
import datetime


In [356]:
## Функция алгоритма, который каждую минуту исполняет заявки по рынку на покупку инструмента с заданным объемом.

def execution_algo(order_required_amount):
    
    ## Таблица по всем сделкам
    all_trades_data = pd.DataFrame(columns=['time(utc timestamp)', 'amount(usd)', 'price', 'side', 'time', 'amount', 'weighted_average_price', 'slippage', 'trading fee'])
    
    ## Первый момент часа, в который мы начинаем торговать.
    current_moment = all_data.timestamp[0].round(freq='S')
    
    total_traded_amount = 0

    
    ## В цикле берём первый момент (первую строку) за каждую минуту и рассчитываем показатели.

    for minute in range(60):
        
        minute_start_data = all_data[all_data['timestamp']>=current_moment][0:1]
    
    
        ## Объём, который мы сможем получить в текущую минуту - это либо весь нужный нам объём заявки, либо весь объём предложения.
        
        total_ask_amount = float(minute_start_data['total_ask_amount'])
        order_traded_amount = min(total_ask_amount, order_required_amount)
       
    
        ## Считаем, до какого ценового уровня нам нужно дойти, чтобы собрать допустимый объём - order_traded_amount.
    
        price_levels_amount_count = 0
        amount_column_num = 0 + 11
    
        while price_levels_amount_count < order_traded_amount:
            amount_column_num = amount_column_num + 1
            price_levels_amount_count = price_levels_amount_count + np.array(minute_start_data)[0,amount_column_num]
         
        final_column_num = amount_column_num
    
    
        ## Считаем, какой объём на последнем ценовом уровне будет лишним.
    
        odd_amount = price_levels_amount_count - order_traded_amount
        odd_weighted_average_price = np.array(minute_start_data)[0,final_column_num-10] * odd_amount / order_traded_amount

    
        ## Цену считаем как средневзвешенная цена в этом трейде + комиссия, уплаченная за 1 единицу объёма (0,07% от средневзвешенной цены).
        ## Объём(в долларах) считаем как средневзвешенная цена умножить на проторгованный в этом трейде объём.
        ## Проскальзывание считаем как разницу средневзвешенной цены и лучшей цены в стакане, поделить на лучшую цену в стакане (величина проскальзывания - в долях).   
    
        weighted_average_price = 0
    
        for amount_column_num in range (12, final_column_num+1):
            weighted_average_price = weighted_average_price + np.array(minute_start_data)[0,amount_column_num-10] * np.array(minute_start_data)[0,amount_column_num] / order_traded_amount
    
        weighted_average_price = weighted_average_price - odd_weighted_average_price
    
        amount_usd = weighted_average_price * order_traded_amount
    
        trading_fee = amount_usd * 0.0007
        trading_fee_per_unit = weighted_average_price * 0.0007
    
        price = weighted_average_price + trading_fee_per_unit
    
        slippage = (weighted_average_price - np.array(minute_start_data)[0,2]) / np.array(minute_start_data)[0,2]
        
    
        ## Собираем все данные по трейду за текущую минуту и добавляем в таблицу по всем сделкам.
    
        minute_trade_data = pd.DataFrame({'time(utc timestamp)': minute_start_data['timestamp.1'],
            'amount(usd)': amount_usd,
            'price': price,
            'side': 'Buy',
            'time': minute_start_data['timestamp'],
            'amount': order_traded_amount,
            'weighted_average_price': weighted_average_price,
            'slippage': slippage,
            'trading fee': trading_fee})
    
        all_trades_data = all_trades_data.append(minute_trade_data)
   

        ## Переходим к первому моменту следующей минуты.
    
        current_moment = current_moment + datetime.timedelta(minutes=1)
    
    
    return all_trades_data
        

In [357]:
## Получение данных из csv и первичная обработка.

all_data = pd.read_csv('tmp.csv')
all_data = pd.DataFrame(all_data)

all_data = all_data.drop(columns=['exchange','symbol','local_timestamp','bids[0].price','bids[1].price','bids[2].price','bids[3].price','bids[4].price','bids[5].price','bids[6].price','bids[7].price','bids[8].price','bids[9].price','bids[0].amount','bids[1].amount','bids[2].amount','bids[3].amount','bids[4].amount','bids[5].amount','bids[6].amount','bids[7].amount','bids[8].amount','bids[9].amount'])

all_data['total_ask_amount'] = all_data.filter(items=['asks[0].amount','asks[1].amount','asks[2].amount','asks[3].amount','asks[4].amount','asks[5].amount','asks[6].amount','asks[7].amount','asks[8].amount','asks[9].amount']).sum(axis=1)

all_data.timestamp = all_data.timestamp.apply(pd.to_datetime)


In [358]:
## Исполняем алгоритм с нужным объёмом заявки.
all_trades_data = execution_algo(1000)

## Количество купленного инструмента (в юнитах). 
total_traded_amount = all_trades_data['amount'].sum()
print('Количество купленного инструмента: ', total_traded_amount)

## Средняя цена покупки (с учётом комиссии).
average_price = all_trades_data['price'].mean()
print('Средняя цена покупки: ', average_price)

## Средний slippage по всем сделкам (в долях).
average_slippage = all_trades_data['slippage'].mean()
print('Средний slippage по всем сделкам: ', f"{average_slippage:.8f}")


Количество купленного инструмента:  60000
Средняя цена покупки:  60928.39671033331
Средний slippage по всем сделкам:  0.00002068


In [359]:
## Формируем итоговый csv-файл.
all_trades_data.index.name = 'id'
all_trades_csv = all_trades_data.drop(columns=['time', 'amount', 'weighted_average_price', 'slippage', 'trading fee'])
all_trades_csv.to_csv('Данные по сделкам - объём 1000.csv')


In [ ]:
## Для проверки.

In [360]:
minute_trade_data

,id,time(utc timestamp),amount(usd),price,side,time,amount,weighted_average_price,slippage,trading fee
11707,11707,1617357540042000,61015500.0,61058.21085,Buy,2021-04-02 09:59:00,1000,61015.5,2.384954e-16,42710.85


In [361]:
all_trades_data

,time(utc timestamp),amount(usd),price,side,time,amount,weighted_average_price,slippage,trading fee
id,,,,,,,,,
0,1617354000424000,6.089250e+07,60935.124750,Buy,2021-04-02 09:00:00.400,1000,60892.500,0.000000e+00,42624.7500
225,1617354060299000,6.085750e+07,60900.100250,Buy,2021-04-02 09:01:00.200,1000,60857.500,0.000000e+00,42600.2500
431,1617354120318000,6.085500e+07,60897.598500,Buy,2021-04-02 09:02:00.300,1000,60855.000,0.000000e+00,42598.5000
607,1617354180146000,6.084600e+07,60888.592200,Buy,2021-04-02 09:03:00.100,1000,60846.000,0.000000e+00,42592.2000
776,1617354240253000,6.082200e+07,60864.575400,Buy,2021-04-02 09:04:00.200,1000,60822.000,0.000000e+00,42575.4000
1017,1617354300042000,6.084300e+07,60885.590100,Buy,2021-04-02 09:05:00.000,1000,60843.000,0.000000e+00,42590.1000
1264,1617354360003000,6.080450e+07,60847.063150,Buy,2021-04-02 09:06:00.000,1000,60804.500,6.578893e-05,42563.1500
1551,1617354420028000,6.082241e+07,60864.980684,Buy,2021-04-02 09:07:00.000,1000,60822.405,6.658775e-06,42575.6835
1761,1617354480273000,6.083520e+07,60877.779637,Buy,2021-04-02 09:08:00.200,1000,60835.195,3.205392e-06,42584.6365
